In [1]:
# reading MERRA2 hourly files, running through a year,
#  and outputting a mean grid at its native resolution as downloaded
# 2/1/18 fixed bug in calculation of daytime-only zonal means
# 5/1/18 now can center on LST with set 'width' and handle season average grids
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap #, cm
import glob

In [2]:
mdir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/MERRA/'
yr = '15' # year 20x
de = 6 # time delta, in hours, around midday/midnight. i.e. 6 means 6-18/18-6, 3 = 9-15/21-3
ctr= 12 # local solar time to center on i.e. noon=12, 1:30pm=13.5
ts = 1 # time step in hrs of data (cant be changed)
mons = sorted(glob.glob(mdir+yr+'*/'))
print(len(mons))
ordr = [11,0,1,2,3,4,5,6,7,8,9,10]
mons = [mons[i] for i in ordr]
#print(mons) # run in order DJF/MMA/JJA/SON/
hrs = str(ctr)+'_'+str(ctr-de)+'to'+str(ctr+de) #'0to24'

12


In [3]:
gsize = 0.5 # in degrees, normal grid
latmax= 90  # NS latitude limits of grid (downloaded this way)
nx, ny = int(360.0/gsize), int(2.0*latmax/gsize) 
hlf = int(nx/2) # MERRA is 0to360, move to -180to180 grid
# MERRA data on slightly different grid than EC, at midpoints rather than edges
mean_grid_iwp = np.zeros([ny, nx])
zonal_mean_iwp = np.zeros(ny)
mean_grid_iwps = np.zeros([ny, nx])
count = 0 # initialize
counts = 0 # initialize
ndays = 0
#seasons = ['DJF','DJF','DJF','MAM','MAM','MAM','JJA','JJA','JJA','SON','SON','SON']

In [4]:
# read in landmask for era5:
#lsfile = ecdir+'land0.5deg.nc' # make sure size matches gsize above
#lf = Dataset(lsfile,'r')
#lmask = np.array(lf['lsm'])[0,:,:] # 0.0 for ocean, 1.0 for land

In [9]:
for m in mons:#[0:1]:
    mo = m[len(m)-3:len(m)-1]
    print(mo)
    iwpall =[]
    ndays = 0
    if mo == '12' or mo == '03' or mo == '06' or mo == '09': #reset for each new season
        mean_grid_iwps = np.zeros([ny, nx])
        mean_grid_swps = np.zeros([ny, nx])
        counts = 0 # initialize
        ndayss = 0
        if mo=='12': seas = 'DJF'
        if mo=='03': seas = 'MAM'
        if mo=='06': seas = 'JJA'
        if mo=='09': seas = 'SON'
    days = sorted(os.listdir(m))
    print('starting mon: ',m) # all daily files in the month:
    for d in days:#[0:1]:
        f = Dataset(m+d,'r') # "total precipitable ice water / total cloud ice water" in kg/m^2
        iwp = np.array(f['TQI']) # "total column ice/snow water" in kg/m^2
        ndays += 1 # count days processed
        ndayss += 1 # count days processed
        for t in range(np.shape(iwp)[0]):
                
            subz = ((ctr+12)-t*ts)/24.0 * nx #)*((t*ts)/24.0) #+ nx 
            sub1, sub2 = int( subz - nx*de/24.0), int( subz + nx*de/24.0)
            #print(sub1,sub2)
            
            if sub1 < 0 and sub2 < 0: sub1, sub2 = sub1+nx, sub2+nx
            if sub1 < 0 or sub2 > nx:
                if sub1 < 0:  sub1 += nx
                if sub2 > nx: sub2 -= nx
                iwp[t,:,sub1:sub2] = np.nan #for zonal means calc
                mean_grid_iwp[:,0:sub2] += iwp[t,:,0:sub2]
                mean_grid_iwp[:,sub1:nx] += iwp[t,:,sub1:nx]
                mean_grid_iwps[:,0:sub2] += iwp[t,:,0:sub2]
                mean_grid_iwps[:,sub1:nx] += iwp[t,:,sub1:nx]
                zonal_mean_iwp[:] += np.nanmean(iwp[t,:,:],axis=1) #dont count 
            else:
                mean_grid_iwp[:,sub1:sub2] += iwp[t,:,sub1:sub2]
                mean_grid_iwps[:,sub1:sub2] += iwp[t,:,sub1:sub2]
                zonal_mean_iwp[:] += iwp[t,:,sub1:sub2].mean(axis=1)
                
            count += 1
            counts += 1
            
            totice = np.reshape(iwp[t,:,:], [ny,nx])
            total_ice = np.reshape(totice[60:300,:],nx*240) # just 60NS into 1D  
            iwpall.extend(total_ice)
    
    if mo == '02' or mo == '05' or mo == '08' or mo == '11': #write out at end of each 3mo season
        mean_grid_iwps /= float(counts/(24.0/(de*2)))
        nds = str(ndayss)
        fin = 'grids/merra.'+seas+'.'+str(nx)+'x.'+str(ny)+'y.'+hrs+'_'+str(ts)+'hrly.iwp.v1.'+yr+'.'+nds
        shift_grid = np.zeros([ny,nx]) #mean_grid_iwp[:,:]
        shift_grid[:,0:hlf] = mean_grid_iwps[:,hlf:nx]
        shift_grid[:,hlf:nx] = mean_grid_iwps[:,0:hlf]
        np.save(fin+'.grid',shift_grid) #mean_grid_iwps)

    nd = str(ndays)
    binz = [2**x for x in range(-14,6)]
    binzz = [-.001,2**(-14)]
    hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
    histaz, bin_edges = np.histogram(np.array(iwpall),bins=binzz)
    print('histaz',histaz,hista)
    fin = 'grids/merra.histo.all.60NS.v2.'+yr+mo+'.'+nd
    #fin = 'grids/era5.histo.all.ciwonly.v2.'+yr+mo+'.'+nds
    outz = np.append(histaz,hista)
    np.save(fin,outz) #hista)
    

mean_grid_iwp /= float(count/(24.0/(de*2)))
zonal_mean_iwp /= float(count)

12
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/MERRA/1512/
histaz [50394207] [3337716 3910991 4633725 5479636 6309624 7195864 8286152 9353801 9700848
 8898683 7066842 3671106  322626    1379       0       0       0       0
       0]
01
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/MERRA/1501/
histaz [52556427] [3572732 4088521 4753717 5554630 6312068 6914374 7678812 8426447 8867743
 8500414 7120164 3894845  321658     648       0       0       0       0
       0]
02
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/MERRA/1502/
histaz [45866099] [3134692 3651547 4290198 5036449 5732769 6462301 7322965 8120234 8486226
 7921642 6391588 3426479  278094     317       0       0       0       0
       0]
03
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/MERRA/1503/
histaz [49404238] [3290685 3873841 4613459 5479356 6379310 7340805 8405368 9341947 9746084
 9101470 7338269 3932043  315574     751       0       0       0       0
      

In [ ]:
# create histogram of IWP
#iwpa = np.array(iwpall)
binz = [2**x for x in range(-14,5)]

#print(iwpa.max(),iwpa.min())
#hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
#histi, bin_edges  = np.histogram(np.array(iwpi),bins=binz)
#hists, bin_edges  = np.histogram(np.array(iwps),bins=binz)

In [ ]:
nd = str(ndays)
# dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
#out = open('grids/merra.'+str(nx)+'x.'+str(ny)+'y.3hrly.iwp.zi.hist.v1.'+yr+'.'+nd+'.bin','wb') #write binary mode
hrs = str(ctr)+'_'+str(ctr-de)+'to'+str(ctr+de) #'0to24'
fin = 'grids/merra.'+str(nx)+'x.'+str(ny)+'y.'+hrs+'_'+str(ts)+'hrly.iwp.v1.'+yr+'.'+nd
shift_grid = np.zeros([ny,nx]) #mean_grid_iwp[:,:]
#hlf = int(nx/2)
shift_grid[:,0:hlf] = mean_grid_iwp[:,hlf:nx]
shift_grid[:,hlf:nx] = mean_grid_iwp[:,0:hlf]
np.save(fin+'.grid',shift_grid)
np.save(fin+'.zonal',zonal_mean_iwp)
#np.save(hista,fin+'.zonal')

In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.plot(binz[1:],hista/sum(hista),'r-o',label='SWP+CIWP')
#plt.plot(binz[1:],histi/sum(histi),'b-o',label='Cloud ice')
#plt.plot(binz[1:],hists/sum(hists),'k-o',label='Snow')
#plt.xscale('log')
#plt.xlabel(r'$IWP [kg/m^2]$')
#plt.ylabel('Fraction of cloudy counts')
#plt.title('Oceanic IWP and SWP, '+mon+' ERA5')#dire)
#plt.ylim(.005,.16)
#plt.legend()
#plt.show()
#fig.savefig('img/era5.'+mon+'.histo.png')

In [ ]:
# plot zonal means
figz = plt.figure(figsize=[7,7])
zax = np.linspace(-1*latmax,latmax,ny)
#plt.plot(1000*zonal_mean_iwp[::-1]+zonal_mean_swp[::-1],zax,'r-o',label='SWP+CIWP')
#plt.plot(1000*zonal_mean_iwp[::-1],zax,'b-o',label='CIWP')
#plt.plot(1000*zonal_mean_swp[::-1],zax,'g-o',label='SWP')
#plt.plot(1000*(zonal_mean_swp[::-1]+zonal_mean_iwp[::-1]),zax,'k-o',label='CIWP+SWP')
plt.plot(zonal_mean_iwp[::-1],zax,'b-o',label='IWP')
#plt.plot(1000*zonal_mean_swp[:],zax,'g-o',label='SWP')
#plt.plot(1000*(zonal_mean_swp[:]+zonal_mean_iwp[:]),zax,'k-o',label='CIWP+SWP')
plt.xlabel(r'$IWP [kg/m^2]$')
plt.ylabel(r'$Latitude [^oN]$')
plt.title('MERRA '+dn+' 20'+yr+' zonal means')
plt.legend()
plt.show()
figz.savefig('img/merra.'+dn+'.20'+yr+'.day.zonal.jpg',dpi=400)

In [ ]:
# plot global mean maps of CIWP/SWP
fig1 = plt.figure(figsize=[11,5])
print(info(mean_grid_iwp))
#grdmap(shift_grid,-180,latmax,.001,0.2,1,'MERRA 20'+yr+' IWP mean')#+mon)
grdmap(mean_grid_iwp,0,latmax,.001,0.2,1,'MERRA '+dn+' 20'+yr+' IWP mean')#+mon)
#grdmap(mean_grid_iwp,lomin=-180,lamin=70,vmin=.001,vmax=0.4,expflag=1,'ERA5 IWP')
fig1.savefig('img/merra.20'+yr+'.'+dn+'.iwp_grid.jpg',dpi=500)
#fig2 = plt.figure(figsize=[11,5])
#print(info(mean_grid_swp))
#grdmap(mean_grid_swp,-180,70,.001,0.2,1,'ERA5 20'+yr+' SWP mean')#+mon)
##grdmap(mean_grid_swp,lomin=-180,lamin=70,vmin=.001,vmax=0.4,expflag=1)
#fig2.savefig('img/era5.20'+yr+'.swp_grid.png',dpi=300)